## Imports ##

In [4]:
import os
import csv
import pandas as pd
import numpy as np
import math
import datetime
from numpy import linalg


#from __future__ import (absolute_import, division, print_function,
#                        unicode_literals)

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
#surprise.prediction_algorithms.matrix_factorization.SVD

## Importing and formating ##

In [5]:
relative_path = os.getcwd() + "/data"

df = pd.read_csv(relative_path + '/data_train.csv').set_index("Id")
movies, people = zip(*[tuple(int(x[1:]) for x in x.split("_")) for x in df.index.values])
df["movies"] = movies
df["people"] = people
df = df.reset_index().drop(columns=["Id"])

predictions = pd.read_csv(relative_path + '/sample_submission .csv').set_index("Id")
toPredict = predictions.copy()
movies, people = zip(*[tuple(int(x[1:]) for x in x.split("_")) for x in toPredict.index.values])
toPredict["movies"] = movies
toPredict["people"] = people
toPredict = toPredict.reset_index().drop(columns=["Id"])



FileNotFoundError: File b'/home/arnaud/Documents/EPFL/MA1/ML/ML-Project-2/data/sample_submission .csv' does not exist

## Training ##

In [39]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['people', 'movies', 'Prediction']], reader)

#Ameliorable
trainset, testset = train_test_split(data, test_size=.000001)

algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
#algo = SVD()
algo.fit(trainset)

submission = toPredict[['people', 'movies', 'Prediction']]
submission = submission.values.tolist()
submission_pred = algo.test(submission)

AttributeError: 'DatasetAutoFolds' object has no attribute 'build_full_train_set'

In [58]:
np.linalg()

TypeError: 'module' object is not callable

In [10]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['people', 'movies', 'Prediction']], reader)
best_accuracy = 1e6
trainset, testset = train_test_split(data, test_size=0.2)

#Parameters testing
factors = np.linspace(50, 175, num = 6)
learning_rates = np.linspace(0.001 , 0.1, num = 5)
regularization_rates = np.linspace(0.001, 0.1, num = 5)
#Besoin d'essayer n_echos?
for factor in factors:
    print(factor)
    print(datetime.datetime.now())
    for learning_rate in learning_rates:
        for regularization_rate in regularization_rates:
            algo = SVD(n_factors=int(factor), lr_all= int(learning_rate), reg_all=int(regularization_rate))
            algo.fit(trainset)
            predictions = algo.test(testset)
            my_accuracy = accuracy.rmse(predictions, verbose=True)
            if my_accuracy < best_accuracy:
                best_set_up = (factor, learning_rate, regularization_rate)
                best_accuracy = my_accuracy

50.0
2018-12-04 14:52:30.467600
RMSE: 1.1211
RMSE: 1.1211
RMSE: 1.1211
RMSE: 1.1211
RMSE: 1.1210
RMSE: 1.1212
RMSE: 1.1212
RMSE: 1.1212
RMSE: 1.1213
RMSE: 1.1211
RMSE: 1.1212
RMSE: 1.1214
RMSE: 1.1212
RMSE: 1.1211
RMSE: 1.1213
RMSE: 1.1215
RMSE: 1.1214
RMSE: 1.1213
RMSE: 1.1209
RMSE: 1.1209
RMSE: 1.1212
RMSE: 1.1209
RMSE: 1.1214
RMSE: 1.1212
RMSE: 1.1212
75.0
2018-12-04 15:02:52.368602
RMSE: 1.1223
RMSE: 1.1222
RMSE: 1.1221
RMSE: 1.1225
RMSE: 1.1224
RMSE: 1.1225
RMSE: 1.1220
RMSE: 1.1222
RMSE: 1.1224
RMSE: 1.1227
RMSE: 1.1223
RMSE: 1.1225
RMSE: 1.1222
RMSE: 1.1222
RMSE: 1.1226
RMSE: 1.1223
RMSE: 1.1220
RMSE: 1.1222
RMSE: 1.1224
RMSE: 1.1221
RMSE: 1.1222
RMSE: 1.1223
RMSE: 1.1222
RMSE: 1.1223
RMSE: 1.1222
100.0
2018-12-04 16:20:49.786869
RMSE: 1.1237
RMSE: 1.1236
RMSE: 1.1231
RMSE: 1.1238
RMSE: 1.1233
RMSE: 1.1233
RMSE: 1.1240
RMSE: 1.1235
RMSE: 1.1227
RMSE: 1.1235
RMSE: 1.1236
RMSE: 1.1233
RMSE: 1.1237
RMSE: 1.1237
RMSE: 1.1232
RMSE: 1.1235
RMSE: 1.1232
RMSE: 1.1231
RMSE: 1.1233
RMSE: 

In [11]:
best_set_up

(50.0, 0.1, 0.025750000000000002)

In [5]:
regularization_rates

array([0.001  , 0.02575, 0.0505 , 0.07525, 0.1    ])

In [12]:
data = Dataset.load_from_df(df[['people', 'movies', 'Prediction']], reader)
trainset = data.build_full_trainset()
algo = SVD(n_factors= 50, lr_all=0.1, reg_all=0.02575 )
algo.fit(trainset)
submission = toPredict[['people', 'movies', 'Prediction']]
submission = submission.values.tolist()
submission_pred = algo.test(submission)
print("User-based Model : Test Set")
accuracy.rmse(submission_pred, verbose=True)

User-based Model : Test Set
RMSE: 1.1562


1.1562460875174616

In [13]:
data = {'Id': ['r' + str(submission_pred[0].iid) + '_c' + str(submission_pred[0].uid)], 'Prediction': [str(submission_pred[0].est)]}
for i in range(1, len(submission_pred)):
    data['Id'].append('r' + str(submission_pred[i].iid) + '_c' + str(submission_pred[i].uid))
    data['Prediction'].append(str(submission_pred[i].est))
submission_df = pd.DataFrame(data)
np.save('sub_SVD', submission_df)

In [38]:
# get RMSE
print("User-based Model : Test Set")
accuracy.rmse(submission_pred, verbose=True)


User-based Model : Test Set
RMSE: 1.0228


1.0228231567908375

In [14]:
submission_df = np.load('sub.npy')
submission_df = pd.DataFrame(submission_df)
submission_df = submission_df.rename(columns={0 : 'Id', 1 : 'Prediction'})
for index, row in submission_df.iterrows():
     row.Prediction = round(float(row.Prediction))
submission_df.to_csv(relative_path + '/submission.csv')

In [62]:
accuracy

<module 'surprise.accuracy' from '/home/arnaud/anaconda3/lib/python3.7/site-packages/surprise/accuracy.py'>